In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_excel('aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21::].to_numpy()

In [3]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(395, 21)

In [4]:
dnn_model = tf.keras.models.load_model('dnn.h5')

In [5]:
def my_loss(real_data, gen_data):
    dist = tf.math.sqrt(tf.reduce_sum(tf.math.square(tf.math.subtract(real_data, gen_data))))
    y_pred = dnn_model(gen_data)

    a = tf.math.subtract(y_pred, 42)
    b = dist

    loss = tf.math.add(a, b)
    return loss

In [6]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(256, input_dim=21, activation='relu'))
generator.add(tf.keras.layers.Dense(128, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(128, activation='relu'))
generator.add(tf.keras.layers.Dense(256, activation='relu'))
generator.add(tf.keras.layers.Dense(128, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='sigmoid'))

generator.compile(loss=my_loss, optimizer='adam')

In [7]:
def return_latest():
    df = pd.read_excel('aug_nine_var.xlsx').iloc[:,1::]
    X = df.iloc[:,0:21]
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    latest = X[-1].reshape(1, 21)

    return tf.cast(latest, dtype=tf.float32)

In [8]:
result = []
pop_result = []
for i in range(1000):
    noise = tf.random.normal([1, 21])
    X_pred = generator.predict(noise, verbose=0)
    real_data = return_latest()
    loss = my_loss(real_data, X_pred)
    generator.train_on_batch(X_pred, real_data)

    pop = dnn_model.predict(X_pred, verbose=0)
    
    result.append(X_pred)
    pop_result.append(pop)

    if i % 10 == 0:
        pass
    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[0.39015293]], pop: [[40.600033]]
epoch: 1, loss: [[0.30639303]], pop: [[40.515266]]
epoch: 2, loss: [[0.28061616]], pop: [[40.488266]]
epoch: 3, loss: [[0.25424016]], pop: [[40.464603]]
epoch: 4, loss: [[0.21562779]], pop: [[40.426846]]
epoch: 5, loss: [[0.20947921]], pop: [[40.42145]]
epoch: 6, loss: [[0.17131925]], pop: [[40.384968]]
epoch: 7, loss: [[0.08358526]], pop: [[40.30046]]
epoch: 8, loss: [[0.01828432]], pop: [[40.235687]]
epoch: 9, loss: [[-0.14348125]], pop: [[40.080334]]
epoch: 10, loss: [[-0.21545446]], pop: [[40.008476]]
epoch: 11, loss: [[-0.53111935]], pop: [[39.699554]]
epoch: 12, loss: [[-0.62814534]], pop: [[39.60368]]
epoch: 13, loss: [[-0.5371554]], pop: [[39.69237]]
epoch: 14, loss: [[-0.81258094]], pop: [[39.419548]]
epoch: 15, loss: [[-2.4684005]], pop: [[37.712093]]
epoch: 16, loss: [[-2.974515]], pop: [[37.16591]]
epoch: 17, loss: [[-3.1358383]], pop: [[36.996338]]
epoch: 18, loss: [[-2.7172222]], pop: [[37.459564]]
epoch: 19, loss: [[-3.2

In [9]:
result = np.array(result).reshape(1000, 21)
pop_result = np.array(pop_result).reshape(1000, 1)
X_result = scaler.inverse_transform(result)

pd_result = pd.DataFrame(X_result)
pd_result.to_excel('basic_formula.xlsx')

pd_pop_result = pd.DataFrame(pop_result)
pd_pop_result.to_excel('basic_formula_pop.xlsx')

In [10]:
error

NameError: name 'error' is not defined

In [ ]:
noise_arr = []
for i in range(4000):
    noise_arr.append(tf.random.normal([1,21]))
noise_arr = np.array(noise_arr)

hist = generator.fit(noise_arr, )